## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_1_last_epoch.h5
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_2_last_epoch.h5
/kaggle/input/145-googleq-a-train-3fold-bert-base-unc-concat/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Concatenate(axis=1)([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.943506,0.702138,0.246625,0.532443,0.442387,0.539606,0.648479,0.647434,0.550600,...,0.914784,0.922670,0.589244,0.975701,0.963735,0.833800,0.015269,0.023995,0.924960,0.913798
1,46,0.890229,0.545138,0.006208,0.810702,0.778357,0.918333,0.539705,0.484191,0.050367,...,0.681994,0.948175,0.635305,0.977282,0.981592,0.888290,0.932702,0.098544,0.047748,0.896776
2,70,0.910535,0.606420,0.022693,0.752721,0.903414,0.960349,0.593662,0.486047,0.128464,...,0.848324,0.940894,0.559056,0.972051,0.970604,0.851584,0.063762,0.043446,0.931042,0.903975
3,132,0.897259,0.470231,0.007588,0.779059,0.744907,0.923108,0.519667,0.459822,0.047279,...,0.702385,0.948859,0.653412,0.968523,0.981404,0.885876,0.816613,0.133759,0.574656,0.898908
4,200,0.939249,0.395855,0.026058,0.880111,0.649529,0.887517,0.665407,0.579374,0.094516,...,0.658190,0.878853,0.596173,0.967926,0.949943,0.787382,0.260854,0.101783,0.611866,0.900218


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.897647,0.613491,0.043951,0.747545,0.805287,0.846460,0.574672,0.497853,0.211676,...,0.812982,0.936945,0.643044,0.968196,0.972213,0.865995,0.508940,0.125779,0.498219,0.916731
std,2812.670060,0.048967,0.143888,0.087373,0.115827,0.124098,0.137688,0.052453,0.080822,0.201585,...,0.086271,0.027316,0.047657,0.011772,0.013533,0.048668,0.342892,0.067431,0.307204,0.022770
min,39.000000,0.738793,0.319949,0.003515,0.199619,0.240718,0.186172,0.462284,0.341283,0.004581,...,0.583928,0.803178,0.471648,0.909013,0.911256,0.670128,0.001904,0.006366,0.010424,0.802707
25%,2572.000000,0.866184,0.484522,0.007331,0.687342,0.765067,0.820597,0.535567,0.446871,0.059411,...,0.743145,0.927332,0.608868,0.962459,0.967648,0.841891,0.141320,0.073092,0.212828,0.904591
50%,5093.000000,0.903462,0.609762,0.012024,0.757468,0.822383,0.895162,0.565471,0.474701,0.127185,...,0.815940,0.943951,0.648127,0.969684,0.975491,0.874515,0.571953,0.128394,0.509876,0.918561
75%,7482.000000,0.935372,0.733698,0.031084,0.827515,0.885869,0.932991,0.609219,0.536839,0.310575,...,0.893028,0.954643,0.676199,0.976315,0.981361,0.898819,0.833369,0.169337,0.770713,0.932949
max,9640.000000,0.979142,0.918299,0.721262,0.976816,0.986684,0.984006,0.748420,0.755015,0.887434,...,0.958967,0.983479,0.763291,0.987693,0.991092,0.970035,0.981604,0.420350,0.991057,0.965077
